In [1]:
import nltk
import numpy as np
import pandas as pd
import tensorflow as ts
import matplotlib.pyplot as plt

In [2]:
#Удаляем NaN поля
df_pos_full = pd.read_csv('comments_positive.csv')
print('Originial', df_pos_full.shape)
df_pos_full.dropna(axis=0, inplace=True)
print('NaN removed',df_pos_full.shape)

df_neg_full = pd.read_csv('comments_negative.csv')
print('Originial', df_neg_full.shape)
df_neg_full.dropna(axis=0, inplace=True)
print('NaN removed',df_neg_full.shape)

Originial (2000000, 15)
NaN removed (1999977, 15)
Originial (2000000, 15)
NaN removed (1999951, 15)


In [3]:
df_pos_full.head()

,id,parent_id,subreddit_id,link_id,text,score,ups,author,controversiality,parent_link_id,parent_text,parent_score,parent_ups,parent_author,parent_controversiality
0,c092j8m,t1_c092gss,t5_2qh2p,t3_8eyy3,This isn't Twitter: try to comment on the arti...,9582,9582,nraustinii,0,t3_8eyy3,Fucking faggot.,-7526,-7526,Glorificus,0
1,c4imcva,t1_c4im948,t5_2qh1i,t3_t0ynr,"Well, it is exactly what it sounds like. It's ...",9531,9531,Lynfect,0,t3_t0ynr,"Elaborate on this cum box, please.",3841,3841,eeeeevil,0
2,c0s4nfi,t1_c0s4lje,t5_2qh1i,t3_cf1n2,"In soviet Russia, bomb disarms you!",8545,8545,CapnScumbone,0,t3_cf1n2,"I don't live in Russia anymore, and I will not...",621,621,shady8x,0
3,c4ini33,t1_c4incln,t5_2qh1i,t3_t0ynr,"""runin for senitur! #YOLO!""",7430,7430,[deleted],0,t3_t0ynr,This just made me realize that future presiden...,4651,4651,drspg99,0
4,c4imgel,t1_c4ima2e,t5_2qh1i,t3_t0ynr,You step motherfucker.,7173,7173,jbg89,0,t3_t0ynr,I have sex with my step mom when my dad isn't ...,4251,4251,audir8,0


In [4]:
df_neg_full.head()

,id,parent_id,subreddit_id,link_id,text,score,ups,author,controversiality,parent_link_id,parent_text,parent_score,parent_ups,parent_author,parent_controversiality
0,c0a2d2p,t1_c0a2cn1,t5_1a8ah,t3_8pr4w,"Na, not really. \n\nI just hate islam and ever...",-2946,-2946,b34nz,0,t3_8pr4w,What goes through the heads of you people? Is...,459,459,[deleted],0
1,c6okok8,t1_c6oaywb,t5_2s8e9,t3_11otij,lol you're some ugly ass white dude,-2724,-2724,letmetellyouhowitis,0,t3_11otij,This is worth noting.\n\nThankfully I don't.,72,72,flowen65,0
2,c3nlalf,t1_c3nijr7,t5_2qzb6,t3_p9a1v,"First of off, its not true, and second off, I ...",-2132,-2132,iamwoodyharrelson,0,t3_p9a1v,I swear this is *(allegedly)* a true story. I...,4028,4028,AndyRooney,0
3,c10nh8q,t1_c10nc34,t5_6,t3_djasj,Who made you reddit police? I will submit what...,-2117,-2117,JimmyJamesincorp,0,t3_djasj,I was on my way over here to bitch and moan ab...,1214,1214,SloaneRanger,0
4,c3nlufk,t1_c3nlcob,t5_2qzb6,t3_p9a1v,We gotta be...i consider my time valuable.,-1962,-1962,iamwoodyharrelson,0,t3_p9a1v,Should change this AMA to AMAAR (Ask Me Anythi...,1405,1405,bersh,0


In [5]:
# Для удобства разбиваем датасет
df_pos = df_pos_full.sample(frac=0.05,random_state=200)
df_neg = df_neg_full.sample(frac=0.05,random_state=200)
df_pos = df_pos.reset_index(drop=True)
df_neg = df_neg.reset_index(drop=True)

print(df_pos.shape)
print(df_neg.shape)

(99999, 15)
(99998, 15)


In [6]:
num_words_pos = df_pos['text'].apply(lambda x: len(x.split()))
num_words_neg = df_neg['text'].apply(lambda x: len(x.split()))

pos_words_mean, pos_words_std = np.mean(num_words_pos), np.std(num_words_pos)
neg_words_mean, neg_words_std = np.mean(num_words_neg), np.std(num_words_neg)

print("Positive stats:", pos_words_mean, pos_words_std)
print("Negative stats:", neg_words_mean, neg_words_std)

Positive stats: 27.603346033460333 57.20243245166202
Negative stats: 32.63535270705414 51.349239623379994


In [7]:
#Удаляем URL и аббревиатуры
def clean(text, stemming=False, stop_words=True):
    import re
    from string import punctuation
    from nltk.stem import SnowballStemmer
    from nltk.corpus import stopwords
    from nltk import word_tokenize
    
    stops = stopwords.words('english')
    
    #Удаление пробелов
    if type(text) != str or text=='':
        return ''
    
    # Удаляем бесполезные конструкции,пунктуацию и стопслова
    urls = r'http(s)*:\/\/(\w|\.)+(\/\w+)*'
    text = re.sub(urls, '', text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are", text)
    text = re.sub("\'ve", " have", text)
    text = re.sub("\'d", " would", text)
    text = re.sub("cant", "can not", text)
    text = re.sub("can\'t", "can not", text)
    text = re.sub("isn\'t", "is not", text)
    text = re.sub("isnt", "is not", text)
    text = re.sub("whats", "what is", text)
    text = re.sub("what\'s", "what is", text)
    text = re.sub("shouldn't", "should not", text, flags=re.IGNORECASE)
    text = re.sub("I'm", "I am", text)
    text = re.sub(":", " ", text)
    text = re.sub("\\n", " ", text)
    text = re.sub('\&', " and ", text)
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    text = ''.join([word for word in text if word not in punctuation]).lower()
    if stemming:
        sno = SnowballStemmer('english')
        text = ''.join([sno.stem[word] for word in text])
    if stop_words:
        text = text.split()
        text = [word for word in text if word not in stops]
        text = ' '.join(text)
    
    return text
    

In [8]:
# Делаем два класса классификитатора
def posneg(number):
    if number > 0:
        return 1
    else:
        return 0

In [9]:
df_pos['text'] = df_pos['text'].apply(clean).astype(str)
df_pos['parent_text'] = df_pos['parent_text'].apply(clean).astype(str)
df_neg['text'] = df_neg['text'].apply(clean).astype(str)
df_neg['parent_text'] = df_neg['parent_text'].apply(clean).astype(str)

df_pos['score'] = df_pos['score'].apply(posneg)
df_pos['parent_score'] = df_pos['parent_score'].apply(posneg)
df_neg['score'] = df_neg['score'].apply(posneg)
df_neg['parent_score'] = df_neg['parent_score'].apply(posneg)

col_write = ['text', 'score', 'ups', 'controversiality', 'parent_text', 'parent_score', 'parent_ups', 'parent_controversiality']
df_pos_clean = df_pos[col_write].copy()
df_neg_clean = df_neg[col_write].copy()
df_pos_clean.dropna(axis=0, inplace=True)
df_neg_clean.dropna(axis=0, inplace=True)
df_pos_clean.to_csv('clean_positive_train.csv', columns=col_write, index=False)
df_neg_clean.to_csv('clean_negative_train.csv', columns=col_write, index=False)